# FX implied PDF around French presidential election in 2022

## Extracting the FX implied PDF from the FX implied volatility surface using Python

April 2022 - Saeed Amen - https://www.cuemacro.com - saeed@cuemacro.com

Before a scheduled event, we obviously don't know the outcome of that event. However, we do know that there will be an event. Regular repeated events include central bank meetings and economic data releases. Less common events, include elections. Given that elections occur so rarely, it can be difficult to "backtest" any strategy to trade around them. We can obviously look at opinion poll to get an idea of the relative likelihood of any particular winner in an election or at betting markets. Another way is to look at financial markets, such as the FX options market, where traders have skin the in the game. The FX options market is also much bigger than any betting markets we might look at. Furthermore, we might choose to compare the probability distribution from the FX options market to what polls or betting markets are saying.

In order to gauge market pricing for an event, we can look at what is priced in FX options markets. Before a large scheduled event, the options market will "bump" up the implied volatility around that event (an event vol add on). The skew can also change. We can extract from the FX volatility surface, the implied PDF of price moves over any option tenor we'd like. The implied PDF, will give us a probability distribution for FX spot. If you're interested in this topic, have a look at the paper Iain Clark and I wrote on the implied PDF around Brexit for GBPUSD at https://www.mdpi.com/2227-9091/5/3/35 and also Iain's FX options book, which describes how to interpolate the FX volatility surface using a number of different techniques at https://www.amazon.co.uk/Foreign-Exchange-Option-Pricing-Practitioners/dp/0470683686

In this note, we'll show how we can extract the FX implied PDF using Python and market data. If you just want to see the resulting PDF plots, skip to the end!

The Python code below:
* downloads all the FX data necessary from Bloomberg using findatapy https://github.com/cuemacro/findatapy 
* fit the FX volatily surface with finmarketpy https://github.com/cuemacro/finmarketpy (& financepy) 
* plot the extracted implied PDF and vol surface. 

You can also try running the code over different dates around the French presidential election to see how the implied PDF can change over time, different currency pairs and different tenors. You can also try running around the Brexit vote and using GBPUSD. Note, that you'll need a Bloomberg Terminal with DAPI in order to download the data. However, if you have another FX data data source, you can just replace the code related to `fetch_market` to read data from your alternative data source.

## How to code it up

Let's do a bunch of imports to start.

In [20]:
# To make sure our Matplotlib plots are put inside the notebook
%matplotlib inline

import copy

import logging
import sys
logging.disable(sys.maxsize)

We import from the findatapy library, which will make it easy to download FX data from Bloomberg.

In [12]:
# For loading market data
from findatapy.market import Market, MarketDataRequest
from findatapy.market import MarketDataGenerator
market = Market(market_data_generator=MarketDataGenerator())
cache_algo = "cache_algo_return"

For fitting the vol surface using finmarketpy (& financepy underneath).

In [13]:
# For constructing and fitting the vol surface 
from finmarketpy.curve.volatility.fxvolsurface import FXVolSurface

Our last imports are for plotting using plotly and chartpy.

In [81]:
# For plotting
import plotly.offline as py_offline

import plotly.io as pio
pio.renderers.default = "notebook" # larger notebook files, but interactive plots
# pio.renderers.default = "svg"
# pio.renderers.default = "png" # smaller notebook files

from chartpy import Chart, Style

chart = Chart(engine='plotly')
style_template = Style(width=900, height=500, scale_factor=-1, plotly_plot_mode="dash")

## Download FX data from Bloomberg and fit FX volatility surface

We write a function `construct_vol_surface`, which downloads the New York closing prices from Bloomberg for FX spot, FX volatility surface, rates etc. for a user specfied date with findatapy. It only take 2 lines of code to download all the necessary data to fit an FX vol surface!

It then uses this data to construct the vol surface with finmarketpy (underneath all the fitting is done using Dominic O'Kane's financepy library at https://github.com/domokane/FinancePy). We then extract the vol surface for strikes which are -15%/15% of the spot price.

In [167]:
def construct_vol_surface(cross, horizon_date):
    # Download the whole all market data for pricing options (vol surface)
    md_request = MarketDataRequest(start_date=horizon_date, finish_date=horizon_date,
                                   data_source='bloomberg', cut='BGN', category='fx-vol-market',
                                   tickers=cross,
                                   cache_algo=cache_algo)

    df = market.fetch_market(md_request)

    fx_vol_surface = FXVolSurface(market_df=df, vol_function_type='BBG', asset=cross)

    fx_vol_surface.build_vol_surface(horizon_date)
    
    spot = df[cross + '.close'][horizon_date]
    low_spot = spot * 0.85
    high_spot = spot * 1.15
    
    # Note for unstable vol surface dates (eg. over Brexit date) you may need to increase tolerance in FinancePy
    # FinFXVolSurface.buildVolSurface method to get it to fill, or choose different vol_function_type (eg. 'CLARK5')
    return df, low_spot, high_spot, fx_vol_surface.extract_vol_surface(
        low_K_pc=low_spot, high_K_pc=high_spot, num_strike_intervals=120)

## Create functions for plotting the implied PDF and vol surface

In [207]:
# Plot implied PDF in strike space (all interpolated)
# x_axis = strike - index
# y_axis = tenor - columns
# z_axis = implied PDF - values
def plot_implied_pdf(df_vol_dict, style_c, low_spot, high_spot, tenor, title="", df_plot=None):
    style = copy.copy(style_c)
    style.title = 'Plotting implied PDF in strike space ' + str(tenor) + ' ' + title
    style.x_axis_range = [low_spot, high_spot]
    
    if df_plot is None:
        df_plot = df_vol_dict['vol_surface_implied_pdf'][tenor]
        
    # Plot the implied PDF for ON only versus strikes
    fig = chart.plot(df_plot, chart_type='line', style=style)
    
    return fig, df_plot

In [208]:
# Plot vol surface in strike space (all interpolated)
# x_axis = strike - index
# y_axis = tenor - columns
# z_axis = implied vol - values
def plot_vol_surface(df_vol_dict, style_c, title="", df_plot=None):
    style = copy.copy(style_c)
    style.title = 'Plotting vol surface ' + title
    
    if df_plot is None:
        df_plot = df_vol_dict['vol_surface_strike_space'].iloc[:, ::-1]
        
    # Plot the implied PDF for ON only versus strikes
    fig = chart.plot(df_plot, chart_type='surface', style=style)
    
    return fig, df_plot

## Run computation for French presidential elections in 2017 and 2022

We'll now download FX data around the 2017 and 2022 French presidential elections. The vol surface will then be fit, and we'll get figures for the vol surface and the implied PDFs. The 2W implied vol date will cover the first round election whilst the 1M implied vol date will cover the second round election in both instances

* For the 2017 election, we'll use 14 Apr 2017 as our reference date  (23 Apr 2017 for the first round election and 07 May 2017 for the second round election). 
* For the 2022 election, we'll use 01 Apr 2022 as our reference date  (10 Apr 2022 for the first round election and 24 Apr 2017 for the second round election). 

In [227]:
date = "14 Apr 2017"; tenor = "2W"; cross = "EURUSD"
df, low_spot, high_spot, df_vol_dict = construct_vol_surface(cross, date)
fig_vol_surface_2017_2W, df_plot = plot_vol_surface(df_vol_dict, style_copy, title=date)
fig_implied_pdf_2017_2W, df_plot = plot_implied_pdf(df_vol_dict, style_copy, low_spot, high_spot, tenor, title=date)

tenor = "1M"
df, low_spot, high_spot, df_vol_dict = construct_vol_surface(cross, date)
fig_vol_surface_2017_1M, df_plot = plot_vol_surface(df_vol_dict, style_copy, title=date)
fig_implied_pdf_2017_1M, df_plot = plot_implied_pdf(df_vol_dict, style_copy, low_spot, high_spot, tenor, title=date)

In [228]:
date = "01 Apr 2022"; tenor = "2W"; cross = "EURUSD"
df, low_spot, high_spot, df_vol_dict = construct_vol_surface(cross, date)
fig_vol_surface_2022_2W, df_plot = plot_vol_surface(df_vol_dict, style_copy, title=date)
fig_implied_pdf_2022_2W, df_plot = plot_implied_pdf(df_vol_dict, style_copy, low_spot, high_spot, tenor, title=date)

tenor = "1M"
df, low_spot, high_spot, df_vol_dict = construct_vol_surface(cross, date)
fig_vol_surface_2022_1M, df_plot = plot_vol_surface(df_vol_dict, style_copy, title=date)
fig_implied_pdf_2022_1M, df_plot = plot_implied_pdf(df_vol_dict, style_copy, low_spot, high_spot, tenor, title=date)

## How does the volatility surface look like before a big event..?

Earlier we talked about how an FX volatility surface will be "bumped" before a big scheduled event, as traders expect more vol around the event (even if they don't know the actual outcome). Below, we plot the vol surface from 01 Apr 2022. We see that the very short dates have implied vol much lower. As soon as we get to dates which cover the first round of the French presidential election, we can see a massive jump. You would obvserve something similar for equity options where they cover scheduled events like earnings annoucements.

In [230]:
fig_vol_surface_2022_2W

## Comparing implied vol only over the first round French presidential elections

Let's now show the implied PDF for the 2W date in 2017 and 2022. The most obvious difference in 2017 versus 2022, is that there is much fatter left hand tail in 2017 (ie. for EURUSD downside in the event of a far right win) compared to 2022. In a sense, we could argue (with a lot of hindsight!) that the left hand tail in 2017 was too big (ie. downside EURUSD options were too expensive), because ultimately, it would have been very unlikely that any candidate would win in the first round.

In [222]:
fig_implied_pdf_2017_2W

In [223]:
fig_implied_pdf_2022_2W

## Comparing implied vol covering both the first and second rounds of the French presidential elections

For the 2017 and 2022, when looking the 1M tenor which covers both the first and second rounds of the French presidential election, we again see a much bigger left hand tail (ie. EURUSD downside) in 2017. 

In [226]:
fig_implied_pdf_2017_1M

In [225]:
fig_implied_pdf_2022_1M

## Conclusion

We have seen that how to calculate the FX implied PDF from the FX implied volatility surface. This type of analysis can be useful in understanding what is priced into the market for large scheduled events.